In [ ]:
import pandas as pd

In [ ]:
drug_administration_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/drug_administrations.xlsx'
moberg_registry_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_registry_kssg.xlsx'

In [ ]:
exclude_short_infusions = True

In [ ]:
drug_administration_df = pd.read_excel(drug_administration_path)
moberg_registry_df = pd.read_excel(moberg_registry_path)

In [ ]:
screening_start = '2018 Nov 07 11:24:38'
screening_end = '2024 Apr 15 23:27'
n_patients_screened = moberg_registry_df['Pat. Nr.'].nunique()
print(f'Screening start: {screening_start}')
print(f'Screening end: {screening_end}')
print(f'Number of patients screened: {n_patients_screened}')

In [ ]:
drug_administration_df = drug_administration_df[drug_administration_df.monitored]

n_patients_with_diclofenac_and_monitoring = drug_administration_df['pat_nr'].nunique()
print(f'Number of patients with diclofenac and monitoring: {n_patients_with_diclofenac_and_monitoring}')

n_patients_before = drug_administration_df['pat_nr'].nunique()
# print patients with exclusion criterium
print(f'Excluding {drug_administration_df[~pd.isna(drug_administration_df["further_exclusion_criterium"])].shape[0]} infusions with {drug_administration_df[~pd.isna(drug_administration_df["further_exclusion_criterium"])]["further_exclusion_criterium"].nunique()} different further exclusion criteria')
# exclude if further_exclusion_criterium is not Nan
drug_administration_df = drug_administration_df[pd.isna(drug_administration_df['further_exclusion_criterium'])]
# print number of patients excluded
print(f'Excluding {n_patients_before - drug_administration_df["pat_nr"].nunique()} patients with further exclusion criterium')

if exclude_short_infusions:
    n_patients_before = drug_administration_df['pat_nr'].nunique()
    drug_administration_df['infusion_duration'] = (pd.to_datetime(drug_administration_df['drug_end']) - pd.to_datetime(drug_administration_df['drug_start'])).dt.total_seconds() / 3600
    print(f'Excluding {drug_administration_df[drug_administration_df["infusion_duration"] <= 1].shape[0]} infusions with duration <= 1h')
    drug_administration_df = drug_administration_df[drug_administration_df['infusion_duration'] > 1]
    print(f'Excluding {n_patients_before - drug_administration_df["pat_nr"].nunique()} patients with infusions with duration <= 1h')

In [ ]:
n_patients = drug_administration_df.pat_nr.nunique()
n_administrations = drug_administration_df.shape[0]
print(f'Number of patients: {n_patients}')
print(f'Number of administrations: {n_administrations}')

In [ ]:
drug_administration_df['duration'] = (drug_administration_df['drug_end'] - drug_administration_df['drug_start']).dt.total_seconds() / 3600
drug_administration_df['duration'].describe()

In [ ]:
drug_administration_df

In [ ]:
# 2988-10-28 00:00:00 in Date of Birth should be 1988-10-28
moberg_registry_df['Date of Birth'] = moberg_registry_df['Date of Birth'].astype(str).str.replace('2988', '1988')
moberg_registry_df['age'] =  pd.to_datetime(moberg_registry_df['Jahr'], format='%Y').dt.year - pd.to_datetime(moberg_registry_df['Date of Birth']).dt.year

In [ ]:
selected_moberg_registry_df = moberg_registry_df[moberg_registry_df['Pat. Nr.'].isin(drug_administration_df.pat_nr)]

In [ ]:
selected_moberg_registry_df.columns

In [ ]:
temp = selected_moberg_registry_df[['Jahr', 'Pat. Nr.', 'Patient First Name', 'Patient Last Name',
       'Medical Record Number', 'Date of Birth','Recording Start Time']]

In [ ]:
# for every patient in selected_moberg_registry_df, get first and last drug administration from drug_administration_df
first_drug_administration = drug_administration_df.groupby('pat_nr').agg({'drug_start': 'min'}).reset_index()
first_drug_administration.columns = ['Pat. Nr.', 'First Drug Administration']
last_drug_administration = drug_administration_df.groupby('pat_nr').agg({'drug_start': 'max'}).reset_index()
last_drug_administration.columns = ['Pat. Nr.', 'Last Drug Administration']

first_last_drug_administration = first_drug_administration.merge(last_drug_administration, on='Pat. Nr.')
# merge with selected_moberg_registry_df
temp = temp.merge(first_last_drug_administration, on='Pat. Nr.')

In [ ]:
temp

In [ ]:
# temp.to_excel('/Users/jk1/Downloads/selected_moberg_registry_for_extraction_df.xlsx', index=False)


In [ ]:
def create_table1(df):
    table1 = pd.DataFrame()
    table1['Number of patients'] = [n_patients]
    table1['Number of drug administrations'] = n_administrations
    table1['Age'] = f'{df.age.median():.0f} ({df.age.quantile(0.25):.0f}-{df.age.quantile(0.75):.0f})'
    table1['Gender (female)'] = f'{df.Gender.value_counts().get("Female", 0)} ({df.Gender.value_counts().get("Female", 0) / n_patients * 100:.0f}%)'
    
    table1['Neurovascular'] = f'{df.Diagnose.value_counts().get("Stroke", 0)} ({df.Diagnose.value_counts().get("Stroke", 0) / n_patients * 100:.0f}%)'
    table1['TBI'] = f'{df.Diagnose.value_counts().get("TBI", 0)} ({df.Diagnose.value_counts().get("TBI", 0) / n_patients * 100:.0f}%)'
    
    table1['ISS'] = f'{df.ISS.median():.0f} ({df.ISS.quantile(0.25):.0f}-{df.ISS.quantile(0.75):.0f})'
    table1['Hunt & Hess'] = f'{df["Hunt & Hess"].median():.0f} ({df["Hunt & Hess"].quantile(0.25):.0f}-{df["Hunt & Hess"].quantile(0.75):.0f})'
    
    return table1.T

In [ ]:
create_table1(selected_moberg_registry_df)